# 🧪 实验：用 Python 玩一玩奇异值分解（SVD）

> 实验目标：  
> 通过 2–3 个小任务，用 **数值实验** 理解 SVD 的几个核心概念：  
> - `A = U Σ V^T` 的数值验证  
> - 低秩近似（rank-k approximation）  
> - 奇异值与 $A^T A$ 特征值的关系  
> 尽量不手推公式，主要靠 Python 观察现象。

---

## 0. 实验环境准备


In [1]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2025)  # 保证可复现

## 任务 1：数值验证 $A = U \Sigma V^T$，并观察正交性

我们先构造一个 非方阵 矩阵（例如 5×3）：

In [2]:
# 构造一个 5×3 的随机矩阵
m, n = 5, 3
A = np.random.randn(m, n)
print("A shape:", A.shape)


A shape: (5, 3)


然后对它做 SVD：

In [4]:
U, S, VT = np.linalg.svd(A, full_matrices=True)
print("U shape:", U.shape)    # 预期 (5, 5)
print("S shape:", S.shape)    # 预期 (3,)
print("VT shape:", VT.shape)  # 预期 (3, 3)


U shape: (5, 5)
S shape: (3,)
VT shape: (3, 3)


## 任务 1 要求:
1. 数值验证重构：
用 U, S, VT 重构出一个矩阵 A_recon，并比较 A 和 A_recon 的差异：

In [5]:
# 提示：S 是一维数组，需要先变成对角矩阵 Σ
Sigma = np.zeros_like(A, dtype=float)  # shape (5, 3)
for i in range(min(m, n)):
    Sigma[i, i] = S[i]

A_recon = U @ Sigma @ VT

# 计算重构误差的 Frobenius 范数
recon_error = np.linalg.norm(A - A_recon, ord="fro")
print("重构误差 ||A - UΣV^T||_F =", recon_error)


重构误差 ||A - UΣV^T||_F = 2.0079677135289497e-15


2. 验证 U 和 V 的“正交性”：

- 计算 U.T @ U，看看是不是接近单位矩阵 $I$；
- 计算 VT @ VT.T，看看是不是接近 $I$。

In [6]:
UU = U.T @ U
VV = VT @ VT.T

print("U^T U =\n", UU)
print("V V^T =\n", VV)

# 可以再看一下和单位矩阵的差异
I_U = np.eye(UU.shape[0])
I_V = np.eye(VV.shape[0])
print("||U^T U - I||_F =", np.linalg.norm(UU - I_U, ord="fro"))
print("||V V^T - I||_F =", np.linalg.norm(VV - I_V, ord="fro"))


U^T U =
 [[ 1.00000000e+00 -2.92849834e-16 -2.34708513e-16 -1.33442536e-16
  -2.61875875e-17]
 [-2.92849834e-16  1.00000000e+00 -9.43435609e-17 -1.75295943e-16
  -2.99525531e-18]
 [-2.34708513e-16 -9.43435609e-17  1.00000000e+00 -8.55616551e-17
  -3.97685333e-17]
 [-1.33442536e-16 -1.75295943e-16 -8.55616551e-17  1.00000000e+00
   1.66878501e-17]
 [-2.61875875e-17 -2.99525531e-18 -3.97685333e-17  1.66878501e-17
   1.00000000e+00]]
V V^T =
 [[ 1.00000000e+00 -1.39494215e-16  1.20885983e-16]
 [-1.39494215e-16  1.00000000e+00  4.40840420e-17]
 [ 1.20885983e-16  4.40840420e-17  1.00000000e+00]]
||U^T U - I||_F = 1.240154647933748e-15
||V V^T - I||_F = 4.1308417084065844e-16


### 💡 提示

* `np.linalg.svd` 返回的是 **`U`、一维奇异值 `S`、`VT`**，要得到 Σ 需要自己构造对角矩阵。

* “正交矩阵”数值上很难做到完全等于 $I$，只要误差接近 $10^{-10} \sim 10^{-12}$ 就可以认为数值上是正交的。

* 如果你改大矩阵尺寸（比如 50×30），看看误差会不会有明显变化。

---

## 任务 2：低秩近似 —— 用前 k 个奇异值还原矩阵

这一题让你体会：

> “SVD 不只是一个分解工具，还是一个**最优的低秩近似工具**。”

我们构造一个 **有明显低秩结构** 的矩阵，例如：  
`A_low = u v^T + 小噪声`，理论 rank 约为 1。


In [7]:
m, n = 40, 20
u = np.random.randn(m, 1)
v = np.random.randn(1, n)
A_clean = u @ v  # 理论上 rank = 1
noise = 0.1 * np.random.randn(m, n)
A = A_clean + noise

print("A shape:", A.shape)


A shape: (40, 20)


对 A 做 SVD：

In [8]:
U, S, VT = np.linalg.svd(A, full_matrices=False)
print("U shape:", U.shape)    # (40, 20)
print("S shape:", S.shape)    # (20,)
print("VT shape:", VT.shape)  # (20, 20)


U shape: (40, 20)
S shape: (20,)
VT shape: (20, 20)


## 👉 任务 2 要求

1. 只用前 k=1 个奇异值做 rank-1 近似：

In [9]:
k = 1
# 取出前 k 个奇异值和对应的 U、V
U_k = U[:, :k]            # shape (m, k)
S_k = S[:k]               # shape (k,)
VT_k = VT[:k, :]          # shape (k, n)

# 构造 Σ_k
Sigma_k = np.diag(S_k)    # shape (k, k)

# rank-k 近似
A_k = U_k @ Sigma_k @ VT_k

# 计算与原矩阵之间的误差
err_full = np.linalg.norm(A - A_clean, ord="fro")
err_k = np.linalg.norm(A - A_k, ord="fro")

print("原始噪声误差 ||A - A_clean||_F =", err_full)
print("rank-1 近似误差 ||A - A_1||_F =", err_k)


原始噪声误差 ||A - A_clean||_F = 2.734912347234197
rank-1 近似误差 ||A - A_1||_F = 2.6583967007987104


2. 比较不同 k 的效果（例如 k = 1, 2, 3）：

In [10]:
ks = [1, 2, 3, 5]
for k in ks:
    U_k = U[:, :k]
    S_k = S[:k]
    VT_k = VT[:k, :]
    Sigma_k = np.diag(S_k)
    A_k = U_k @ Sigma_k @ VT_k
    err_k = np.linalg.norm(A - A_k, ord="fro")
    print(f"k = {k:2d},  近似误差 ||A - A_k||_F = {err_k:.4f}")


k =  1,  近似误差 ||A - A_k||_F = 2.6584
k =  2,  近似误差 ||A - A_k||_F = 2.4745
k =  3,  近似误差 ||A - A_k||_F = 2.2843
k =  5,  近似误差 ||A - A_k||_F = 1.9623


### 💡 提示

* `full_matrices=False` 会返回“经济版” SVD，更适合做低秩近似。

* SVD 的一个经典性质：**在所有 rank-k 矩阵中，$A_k$ 是离 A 最近的**（对 Frobenius 范数而言）。这里我们不证明，只让你通过数值看“大约是这样”。

* 你也可以画一画奇异值随索引的变化（`plt.plot(S)`），看看为什么 rank-1 就已经很好。


# 🧪 实验：用 Python 玩转一元导数、偏导、梯度 & 矩阵梯度

---

## 0. 实验环境与数据准备

本次实验统一用 Python + NumPy（可选：Matplotlib 画图）。

```python
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2025)  # 固定随机种子，保证每个人结果一致

```
### 我们用一个“简化房价预测”的玩具数据集：
- 输入特征：面积、房龄、距离地铁（可按需要标准化）
- 输出：房价（单位可以随意理解）



In [22]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(2025)  # 保证可复现
# 数据规模可以根据机器性能适当调节
n_samples = 50   # 样本个数
n_features = 3   # 特征维度（面积 / 房龄 / 距地铁）

# 生成特征矩阵 X
X = np.random.rand(n_samples, n_features)  # [0,1) 随机数，可理解为已简单缩放的特征

# 人为设定一个“真实”参数，用来生成标签 y_true
w_true = np.array([3.0, -1.5, -2.0])   # 每个特征的权重
b_true = 5.0                           # 偏置
noise = 0.3 * np.random.randn(n_samples)  # 高斯噪声

y = X @ w_true + b_true + noise        # 房价（目标变量）

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (50, 3)
y shape: (50,)


## 任务 1：一元函数的导数（5.1）

这里我们先固定多维参数，只让 **一个标量参数 $w_{\text{scalar}}$** 变化，观察一元函数的导数。

### 1.1 定义一元损失函数

我们构造一个最简单的一元模型：

$$
\hat{y} = w_{\text{scalar}} \cdot x_1
$$

只用第一列特征 $x_1$ 来预测 $y$。

损失函数（均方误差）定义为：

$$
L(w_{\text{scalar}}) 
= \frac{1}{n} \sum_{i=1}^n \bigl(w_{\text{scalar}} x_{1,i} - y_i\bigr)^2
$$

---

### 👉 任务

1. **写一个 Python 函数**  
   `loss_scalar(w_scalar, x1, y)`  
   输入标量 `w_scalar`，返回标量损失值。

2. **写一个数值导数函数**  
   `numerical_derivative_1d(f, w, h=1e-5)`  
   对任意一元函数 $f(w)$ 计算近似导数：

   $$
   f'(w) \approx \frac{f(w+h) - f(w-h)}{2h}
   $$

3. **选取若干个 $w_{\text{scalar}}$**，计算对应的 $L(w)$ 和 $L'(w)$，观察符号与大小。

---

### 👉 提示

- `x1` 是一维数组（shape: `(n_samples,)`），可用 `X[:, 0]` 提取。
- 数值导数建议使用 **中心差分**（精度更好）。
- 尝试打印不同 `w_scalar` 下的 $(L(w), L'(w))$：
  - 当 $w_{\text{scalar}}$ 很小时，导数一般是正还是负？
  - 当 $w_{\text{scalar}}$ 接近最优值时，是否出现 $L'(w) \approx 0$？


In [23]:
x1 = X[:, 0]

def loss_scalar(w_scalar, x1, y):
    """
    一元参数 w_scalar 的均方误差损失函数。
    输入：
        w_scalar: 标量
        x1: shape (n_samples,)
        y: shape (n_samples,)
    输出：
        标量损失值
    """
    n = len(x1)  # 样本数
    y_hat = w_scalar * x1  # 预测值
    error = y_hat - y      # 误差项
    loss = np.sum(error ** 2) / n  # 均方误差
    return loss

def numerical_derivative_1d(f, w, h=1e-5):
    """
    对一元函数 f(w) 做数值求导。
    f: 只接受一个标量 w 作为输入的函数
    w: 标量
    h: 差分步长
    """
    return (f(w + h) - f(w - h)) / (2 * h)


# TODO: 选几个 w_scalar 点，观察 L(w) 和 L'(w)
for w_scalar in [-3.0, -1.0, 0.0, 1.0, 3.0, 5.0]:
    f_val = loss_scalar(w_scalar, x1, y)
    f_grad = numerical_derivative_1d(lambda w: loss_scalar(w, x1, y), w_scalar)
    print(f"w = {w_scalar:5.2f}, L(w) = {f_val:8.4f}, L'(w) ≈ {f_grad:8.4f}")


w = -3.00, L(w) =  43.8750, L'(w) ≈  -7.3598
w = -1.00, L(w) =  30.5055, L'(w) ≈  -6.0097
w =  0.00, L(w) =  24.8334, L'(w) ≈  -5.3346
w =  1.00, L(w) =  19.8363, L'(w) ≈  -4.6595
w =  3.00, L(w) =  11.8674, L'(w) ≈  -3.3094
w =  5.00, L(w) =   6.5988, L'(w) ≈  -1.9592


> ### ✅ 思考
>
> - 当 $L'(w) < 0$ 时，如果你向右（增大 $w$）走一步，损失是变大还是变小？
> - 这和“导数是切线斜率”有什么关系？

---

## 任务 2：多变量偏导与梯度（5.2）

现在回到完整的三维参数向量 $w \in \mathbb{R}^3$，模型变为：

$$
\hat{y} = X w
$$

损失函数为：

$$
L(w) = \frac{1}{n}\,\|Xw - y\|^2
$$

这里我们不用手动推偏导，而是用 **数值方法估计梯度向量**：

$$
\nabla_w L(w)
\approx
\begin{bmatrix}
\frac{\partial L}{\partial w_1} \\
\frac{\partial L}{\partial w_2} \\
\frac{\partial L}{\partial w_3}
\end{bmatrix}
$$

---

### 2.1 定义多变量损失函数

### 👉 任务

1. 写一个函数 `loss_vec(w, X, y)`  
   其中 `w` 是 shape `(n_features,)` 的一维向量。

2. 写一个函数  
   `numerical_gradient(f, w, h=1e-5)`  
   对多元函数 $f(w)$ 做数值梯度估计：

   $$
   \frac{\partial L}{\partial w_j}
   \approx
   \frac{L(w + h e_j) - L(w - h e_j)}{2h}
   $$

   其中 $e_j$ 是第 $j$ 个坐标方向的单位向量。

---

### 👉 提示

- `w` 是一维数组，复制时请使用 `w.copy()`。
- 在 `numerical_gradient` 中，可以使用 `for j in range(len(w)):` 逐元素计算。
- 注意 **不要在循环中直接修改原始 `w`**，每次都从 `w.copy()` 出发，避免串扰。


In [24]:
def loss_vec(w: np.ndarray, X: np.ndarray, y: np.ndarray) -> float:
    """
    多维参数 w 的均方误差损失。

    w: shape (n_features,)
    X: shape (n_samples, n_features)
    y: shape (n_samples,)

    返回：
        标量 L(w) = 1/n * ||Xw - y||^2
    """
    y_hat = X @ w 
    
    residuals = y_hat - y
    
    loss = np.mean(residuals**2)
    
    return loss


def numerical_gradient(f, w: np.ndarray, h: float = 1e-5) -> np.ndarray:
    """
    对多元函数 f(w) 进行数值梯度估计（中心差分）。

    f : 接受一维向量 w (shape: (d,)) 并返回标量的函数
    w : 当前点，一维向量
    h : 差分步长
    """
    w = w.astype(float)  # 确保是浮点数
    grad = np.zeros_like(w)
    for j in range(len(w)):
        w_plus = w.copy()
        w_minus = w.copy()
        w_plus[j] += h
        w_minus[j] -= h
        f_plus = f(w_plus)
        f_minus = f(w_minus)
        grad[j] = (f_plus - f_minus) / (2.0 * h)
    return grad


print("任务 2：多维梯度示例")

# 随机初始化一个 w0 作为起点
w0 = np.random.randn(n_features)

grad_w0 = numerical_gradient(lambda w: loss_vec(w, X, y), w0)
print("w0:", w0)
print("∇L(w0) ≈", grad_w0)


任务 2：多维梯度示例
w0: [0.95645575 0.47044199 0.75540948]
∇L(w0) ≈ [-4.15691833 -3.35352672 -2.79736823]


> ## 2.2 用梯度做一步“梯度下降”
>
> **任务：**
>
> 1. 计算当前参数 $w_0$ 的损失值 $L_0$。
>
> 2. 做一步梯度下降：
>
>    $$
>    w_{\text{new}} = w_0 - \eta \,\nabla_w L(w_0)
>    $$
>
>    其中 $\eta$ 是学习率（例如 $\eta = 0.1$）。
>
> 3. 观察新参数 $w_{\text{new}}$ 对应的损失值是否下降。
>
> ---
>
> **提示：**
>
> - 学习率不要太大，否则可能出现“震荡”或“发散”。  
>   可先尝试 $\eta = 0.1$ 或 $\eta = 0.01$。
>
> - 如果损失没有下降，可以尝试进一步减小 $\eta$。


In [42]:
eta = 0.58  # 可以尝试不同学习率

L0 = loss_vec(w0, X, y)
w_new = w0 - eta * grad_w0
L_new = loss_vec(w_new, X, y)

print("L(w0)     =", L0)
print("L(w_new)  =", L_new)


L(w0)     = 15.94180542388629
L(w_new)  = 4.169880406137144


> ## ✅ 思考
>
> - 梯度向量的方向，是不是大致指向“损失增大的方向”？
> - 那么 $-\nabla L$ 又表示什么方向？

---

## 任务 3：向量值函数与 Jacobian（5.3）

我们定义一个 **向量值函数**（输出是一个向量）：

$$
g(w) = Xw - y \in \mathbb{R}^n
$$

每个分量为：

$$
g_i(w) = x_i^\top w - y_i
$$

在数值上，我们可以计算 $g(w)$ 的 Jacobian 矩阵：

$$
J_g(w) 
= 
\left[
\frac{\partial g_i}{\partial w_j}
\right]
\in \mathbb{R}^{n \times d}
$$

---

### 3.1 用数值方法估计 Jacobian

### 👉 任务

1. 实现 `g(w, X, y)`，返回 shape `(n_samples,)` 的向量。

2. 实现 `numerical_jacobian(g, w, h=1e-5)`，返回 Jacobian，shape 为  
   `(n_samples, n_features)`：

   - 对每个维度 $j$，对 $w$ 做 $+h$ 与 $-h$ 的扰动；
   - Jacobian 的第 $j$ 列对应 $\dfrac{\partial g}{\partial w_j}$。

---

### 👉 提示

- 这一步几乎与前面的 `numerical_gradient` 相同，只是现在  
  $f(w)$ 从标量变成了向量函数 $g(w)$。
- 你可以先理解一维情况：  
  如果 `g(w)` 返回一个标量，那么 Jacobian 就退化为 **一行向量**。


In [43]:
def g(w, X, y):
    """
    g(w) = Xw - y
    w: shape (n_features,)
    X: shape (n_samples, n_features)
    y: shape (n_samples,)
    返回: shape (n_samples,)
    """
    # TODO: 实现向量值函数
    return X @ w - y

def numerical_jacobian(g_fun, w: np.ndarray, X: np.ndarray, y: np.ndarray,
                       h: float = 1e-5) -> np.ndarray:
    """
    对向量值函数 g(w) 数值估计 Jacobian 矩阵。

    g_fun: g_fun(w, X, y) -> shape (n_samples,) 的函数
    w    : shape (d,)
    返回:
        J: shape (n_samples, d), 其中 J[i, j] ≈ ∂g_i/∂w_j
    """
    w = w.astype(float)
    n = X.shape[0]
    d = w.shape[0]
    J = np.zeros((n, d))

    for j in range(d):
        w_plus = w.copy()
        w_minus = w.copy()
        w_plus[j] += h
        w_minus[j] -= h

        g_plus = g_fun(w_plus, X, y)
        g_minus = g_fun(w_minus, X, y)

        # 对所有样本的第 j 维偏导，一次性计算
        J[:, j] = (g_plus - g_minus) / (2.0 * h)

    return J

w_test = np.random.randn(n_features)
Jg = numerical_jacobian(g, w_test, X, y)
print("Jacobian g(w) shape:", Jg.shape)  # 预期: (n_samples, n_features)


Jacobian g(w) shape: (50, 3)


## 3.2 用 Jacobian 验证链式法则（数值版）

我们知道损失函数可以写成：

$$
L(w) = \frac{1}{n}\,\|g(w)\|^2 
     = \frac{1}{n}\, g(w)^\top g(w)
$$

根据链式法则可得：

$$
\nabla_w L(w)
= \frac{2}{n}\, J_g(w)^\top g(w)
$$

其中：

- $g(w) = Xw - y$  
- $J_g(w)$ 是 $g(w)$ 关于 $w$ 的 Jacobian  
- $n$ 是样本数

---

### 👉 任务：

1. 使用上一步算出的 `Jg` 和 `g(w_test, X, y)`，构造：

   ```python
   grad_from_jacobian = (2.0 / n_samples) * Jg.T @ g(w_test, X, y)
   ```

2. 将它和前面 `numerical_gradient` 计算得到的

   ```python
   grad_direct = numerical_gradient(lambda w: loss_vec(w, X, y), w_test)
   ```

   作比较，看差值是否很小。

---

### 👉 提示：

- 由于都是数值近似，允许有微小误差，可以看：

  ```python
  np.linalg.norm(grad_from_jacobian - grad_direct)
  ```

- 如果误差较大，考虑：

  * 是否步长 $h$ 设得太大或太小？
  * Jacobian 的维度是否写反？


In [44]:
grad_direct = numerical_gradient(lambda w: loss_vec(w, X, y), w_test)
g_val = g(w_test, X, y)
grad_from_jacobian = (2.0 / n_samples) * Jg.T @ g_val

print("grad_direct       :", grad_direct)
print("grad_from_jacobian:", grad_from_jacobian)
print("difference norm   :", np.linalg.norm(grad_direct - grad_from_jacobian))


grad_direct       : [-5.51819791 -4.60104626 -4.06470395]
grad_from_jacobian: [-5.51819791 -4.60104626 -4.06470395]
difference norm   : 1.87526302365957e-10


## 任务 4：矩阵参数的梯度（5.4）

现在参数不再是一个向量，而是一个矩阵 $W \in \mathbb{R}^{d \times k}$。  
假设我们要预测的不止一个“房价”，而是 $k$ 个不同的目标（例如不同评估指标）。

模型：

$$
\hat{Y} = X W \quad (\hat{Y} \in \mathbb{R}^{n \times k})
$$

损失函数：

$$
L(W) = \frac{1}{n}\,\|XW - Y\|_F^2
$$

其中 $\|\cdot\|_F$ 是 Frobenius 范数（所有元素平方和再开方），平方后就是所有元素平方和。

为方便实验，我们可以用已有的 $y$ 复制几列生成 $Y$：


In [49]:
k_outputs = 3  # 输出维度（可以先设为 2）
#Y = np.stack([y, 2*y + 1.0], axis=1)  # shape: (n_samples, 2)
Y = np.stack([y, 2*y + 1.0, -y + 3.0], axis=1)  # shape: (n_samples, 3)


print("Y shape:", Y.shape)


Y shape: (50, 3)


### 4.1 定义矩阵参数的损失函数

👉 **任务：**

1. 实现 `loss_matrix(W, X, Y)`，其中：

   * `W` shape: `(n_features, k_outputs)`
   * `X` shape: `(n_samples, n_features)`
   * `Y` shape: `(n_samples, k_outputs)`

2. 返回标量损失值。

---

👉 **提示：**

* 先计算 `pred = X @ W`，shape 为 `(n_samples, k_outputs)`。

* 误差 `E = pred - Y`，Frobenius 范数平方可以用：

  $$
  \|E\|_F^2 = \sum_{i,j} E_{ij}^2
  $$

  在代码中可写作：

  ```python
  np.sum(E**2)
  ```


In [50]:
def loss_matrix(W, X, Y):
    """
    矩阵参数 W 的多输出均方误差损失。
    W: shape (n_features, k_outputs)
    X: shape (n_samples, n_features)
    Y: shape (n_samples, k_outputs)
    """
    Y_hat = X @ W
    
    E = Y_hat - Y
    sum_squared_error = np.sum(E**2)
    n = X.shape[0]
    loss = (1.0 / n) * sum_squared_error
    
    return loss

# 随机初始化一个 W0
W0 = np.random.randn(n_features, k_outputs)
L_W0 = loss_matrix(W0, X, Y)
print("L(W0) =", L_W0)


L(W0) = 139.6327798181964


### 4.2 数值估计矩阵梯度

矩阵梯度 $\nabla_W L$ 的 shape 应该与 $W$ 一样： `(n_features, k_outputs)`。  
我们仍然用中心差分估计每个元素的偏导：

$$
\frac{\partial L}{\partial W_{ij}}
\approx
\frac{L(W + h E_{ij}) - L(W - h E_{ij})}{2h}
$$

其中 $E_{ij}$ 是一个只有第 $(i,j)$ 个元素为 1，其余为 0 的矩阵。

---

👉 **任务：**

1. 实现 `numerical_gradient_W(f, W, h=1e-5)`，对矩阵参数做数值梯度估计。

2. 打印 `grad_W` 的 shape，确认与 `W0` 一致。

3. 做一次梯度下降：  
   `W_new = W0 - eta * grad_W`  
   观察损失是否下降。

---

👉 **提示：**

- 可以用双重 for 循环（i, j）逐个元素做差分。

- 注意每次需要从 `W` 做一个拷贝 `W_plus = W.copy()`，再对其中一个元素做修改。

- 尽量保持代码清晰，不必过度优化性能（维度很小，运行时间可接受）。


In [51]:
def numerical_gradient_W(f, W: np.ndarray, h: float = 1e-4) -> np.ndarray:
    """
    对矩阵参数 W 进行数值梯度估计（中心差分）。

    f : 接受矩阵 W (shape: (n_features, k_outputs)) 并返回标量的函数
    W : 当前点
    h : 差分步长
    """
    W = W.astype(float)
    grad = np.zeros_like(W)
    rows, cols = W.shape

    for i in range(rows):
        for j in range(cols):
            W_plus = W.copy()
            W_minus = W.copy()

            W_plus[i, j] += h
            W_minus[i, j] -= h

            f_plus = f(W_plus)
            f_minus = f(W_minus)

            grad[i, j] = (f_plus - f_minus) / (2.0 * h)

    return grad

grad_W0 = numerical_gradient_W(lambda W: loss_matrix(W, X, Y), W0)
print("grad_W0 shape:", grad_W0.shape)

eta = 0.1
L_W0 = loss_matrix(W0, X, Y)
W_new = W0 - eta * grad_W0
L_Wnew = loss_matrix(W_new, X, Y)

print("L(W0)   =", L_W0)
print("L(Wnew) =", L_Wnew)


grad_W0 shape: (3, 3)
L(W0)   = 139.6327798181964
L(Wnew) = 106.90957709134454


## 思考：

- 你得到的 grad_W0 中，每一行对应什么含义？每一列呢？

- 如果维度变大（例如 1000×100），你觉得还会有人用“一个元素一个元素”去算梯度吗？现实中的深度学习框架是如何利用矩阵结构一次性计算梯度的？

1. grad_W0 行与列的含义行 (Rows) → 输入特征：第 $i$ 行对应第 $i$ 个输入特征（如面积、房龄）。它反映了该特征如何影响所有的输出结果。列 (Columns) → 输出目标：第 $j$ 列对应第 $j$ 个输出预测值（如房价 A、房价 B）。它反映了为了优化第 $j$ 个预测任务，各特征权重应该如何调整。

2. 维度变大时怎么办？绝对不会用“一个元素一个元素”的数值微分。对于 $1000 \times 100$ 的矩阵，这意味着要运行 10 万次前向传播，计算成本不可接受。深度学习框架的做法：它们利用 解析解公式（如 $\nabla L = \frac{2}{n} X^T (XW - Y)$）和 反向传播 (Backprop)。本质：将几十万次标量运算转换为 一次矩阵乘法。实现：底层调用高度优化的线性代数库（如 BLAS, cuBLAS），在 GPU 上并行计算，瞬间完成所有梯度的推导。
